In [18]:
import os
import pandas as pd
import numpy as np


In [19]:
train_df=pd.read_csv("train_plant_seedling.csv",index_col=False)
test_df=pd.read_csv("test_plant_seedling.csv",index_col=False)

In [20]:
train_df.head()

,Labels,filenames
0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0050f38b3.png
1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0183fdf68.png
2,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0260cffa8.png
3,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",05eedce4d.png
4,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",075d004bc.png


In [21]:
test_df.head()

,Unnamed: 0,filenames
0,0,0021e90e4.png
1,1,003d61042.png
2,2,007b3da8b.png
3,3,0086a6340.png
4,4,00c47e980.png


In [22]:
img_w = 400
img_h = 400
img_c = 3   #bgr
img_size = (img_w , img_h)

In [23]:
from keras.models import Sequential
from keras.layers import Conv2D , MaxPooling2D, Dropout, Flatten,Dense,Activation,BatchNormalization

In [48]:
model = Sequential()

model.add(Conv2D(32,(3,3),activation='relu',input_shape=(img_w,img_h,img_c)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.20))

model.add(Conv2D(64,(3,3),activation='relu',))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.20))

model.add(Conv2D(128,(3,3),activation='relu',))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.20))

model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

# should contain no of categories we are classifing -- here 12
model.add(Dense(12,activation='softmax'))

model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'rmsprop',
    metrics = ['accuracy']
)




In [49]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 398, 398, 32)      896       
                                                                 
 batch_normalization_16 (Bat  (None, 398, 398, 32)     128       
 chNormalization)                                                
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 199, 199, 32)     0         
 g2D)                                                            
                                                                 
 dropout_16 (Dropout)        (None, 199, 199, 32)      0         
                                                                 
 conv2d_13 (Conv2D)          (None, 197, 197, 64)      18496     
                                                                 
 batch_normalization_17 (Bat  (None, 197, 197, 64)    

In [50]:
from keras.callbacks import EarlyStopping,ReduceLROnPlateau

earlystopping = EarlyStopping(patience=10)
reduceLR = ReduceLROnPlateau(monitor = "val_acc",patience =2 , verbose =1 ,factor = 0.5 ,min_lr = 0.00001 )
callbacks = [earlystopping,reduceLR]

In [51]:
from sklearn.model_selection import  train_test_split

train , validate = train_test_split(train_df,test_size=0.25,random_state=42)


In [52]:
train_df.head()

,Labels,filenames
0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0050f38b3.png
1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0183fdf68.png
2,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0260cffa8.png
3,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",05eedce4d.png
4,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",075d004bc.png


In [53]:
# TypeError: If class_mode="categorical", y_col="Labels" column values must be type string, list or tuple.

for l in range(len(train_df["Labels"])):
    lst = [0,0,0,0,0,0,0,0,0,0,0,0]
    # print(train_df["Labels"][l])
    for i in range(12):
        # print(lst)
        if i==train_df["Labels"][l]:
            lst[i]=1
            # print(lst)
            train_df["Labels"]=np.tile(lst,(l,1)).tolist()
train_df

,Labels,filenames
0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0050f38b3.png
1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0183fdf68.png
2,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0260cffa8.png
3,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",05eedce4d.png
4,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",075d004bc.png
...,...,...
4745,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",fc293eacb.png
4746,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",fc441208c.png
4747,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",fed9406b2.png
4748,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",fef5e7066.png


In [54]:
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical,load_img

batch_size = 15
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_gen = train_datagen.flow_from_dataframe(
    train,
    "./train_plant_all/",
    x_col="filenames",
    y_col="Labels",
    target_size=img_size,
    class_mode ='categorical',
    batch_size=batch_size
)

print("train data generator")
validate_datagen = ImageDataGenerator(rescale=1./255)

validate_gen = validate_datagen.flow_from_dataframe(
    validate,
    "./train_plant_all/",
    x_col="filenames",
    y_col="Labels",
    target_size=img_size,
    class_mode=None,
    batch_size=batch_size
)
print("Validate data generator")
test_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)
test_gen = test_datagen.flow_from_dataframe(
    test_df,
    "./test_plant/",
    x_col="filenames",
    target_size=img_size,
    class_mode=None,
    batch_size=batch_size
)

print("test data generator")

Found 3562 validated image filenames belonging to 12 classes.
train data generator
Found 1188 validated image filenames.
Validate data generator
Found 794 validated image filenames.
test data generator


In [55]:
epoch = 10
history = model.fit_generator(
    train_gen,
    epochs=epoch,
    validation_data=validate_gen,
    validation_steps=(validate.shape[0])//batch_size,
    steps_per_epoch= (train.shape[0])//batch_size,
    callbacks=callbacks
)

C:\Users\Janak\AppData\Local\Temp\ipykernel_40416\102994368.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/10
237/237 [==============================] - 1062s 4s/step - loss: 2.1587 - accuracy: 0.4113 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 2/10
237/237 [==============================] - 1031s 4s/step - loss: 1.3535 - accuracy: 0.5887 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 3/10
237/237 [==============================] - 1035s 4s/step - loss: 1.0481 - accuracy: 0.6696 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 4/10
237/237 [==============================] - 4050s 17s/step - loss: 0.9053 - accuracy: 0.7090 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 5/10
237/237 [==============================] - 3798s 16s/step - loss: 0.7620 - accuracy: 0.7432 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 6/10
237/237 [==============================] - 1138s 5s/step - loss: 0.6544 - accuracy: 0.7917 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 7/

In [56]:
model.save("plant_seedling.h5")

In [58]:
prediction = model.predict_generator(test_gen,len(test_gen.filenames))

C:\Users\Janak\AppData\Local\Temp\ipykernel_40416\370301731.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  prediction = model.predict_generator(test_gen,len(test_gen.filenames))


In [60]:
pred=[np.argmax(i) for i in prediction]
pred

[6,
 8,
 5,
 8,
 3,
 5,
 5,
 5,
 5,
 4,
 1,
 5,
 5,
 0,
 5,
 9,
 8,
 3,
 6,
 1,
 8,
 5,
 3,
 5,
 5,
 5,
 5,
 5,
 1,
 1,
 5,
 0,
 5,
 5,
 5,
 5,
 4,
 4,
 5,
 0,
 8,
 5,
 1,
 9,
 5,
 5,
 5,
 2,
 8,
 5,
 5,
 5,
 5,
 5,
 1,
 5,
 4,
 9,
 6,
 5,
 10,
 9,
 5,
 4,
 8,
 6,
 8,
 8,
 5,
 5,
 5,
 8,
 5,
 5,
 5,
 5,
 2,
 4,
 1,
 5,
 5,
 5,
 7,
 9,
 6,
 5,
 0,
 5,
 5,
 5,
 5,
 5,
 5,
 9,
 5,
 5,
 10,
 6,
 5,
 1,
 5,
 5,
 5,
 8,
 5,
 5,
 5,
 5,
 5,
 8,
 1,
 5,
 2,
 5,
 2,
 9,
 5,
 2,
 5,
 5,
 9,
 4,
 8,
 9,
 5,
 5,
 8,
 5,
 3,
 1,
 0,
 3,
 5,
 3,
 5,
 5,
 5,
 0,
 1,
 9,
 5,
 1,
 5,
 8,
 5,
 0,
 5,
 5,
 9,
 5,
 1,
 9,
 5,
 5,
 8,
 8,
 9,
 5,
 1,
 5,
 9,
 5,
 3,
 9,
 5,
 0,
 8,
 1,
 1,
 3,
 5,
 8,
 5,
 9,
 5,
 5,
 2,
 3,
 8,
 5,
 8,
 5,
 9,
 5,
 5,
 5,
 9,
 9,
 8,
 5,
 1,
 2,
 8,
 9,
 5,
 5,
 1,
 5,
 5,
 5,
 5,
 6,
 5,
 5,
 5,
 3,
 6,
 5,
 8,
 9,
 5,
 9,
 3,
 2,
 2,
 5,
 9,
 8,
 8,
 9,
 8,
 2,
 5,
 5,
 5,
 8,
 9,
 8,
 5,
 0,
 5,
 11,
 5,
 5,
 5,
 8,
 1,
 9,
 9,
 1,
 5,
 5,
 5,
 5,
 2,
 6,
 5,
 8,
 4,
 

In [62]:
# replacing with string 
lst = os.listdir("train_plant")
for i in range(len(pred)) :
    pred[i] =  lst[pred[i]]
pred


['Loose Silky-bent',
 'Scentless Mayweed',
 'Fat Hen',
 'Scentless Mayweed',
 'Common Chickweed',
 'Fat Hen',
 'Fat Hen',
 'Fat Hen',
 'Fat Hen',
 'Common wheat',
 'Charlock',
 'Fat Hen',
 'Fat Hen',
 'Black-grass',
 'Fat Hen',
 'Shepherds Purse',
 'Scentless Mayweed',
 'Common Chickweed',
 'Loose Silky-bent',
 'Charlock',
 'Scentless Mayweed',
 'Fat Hen',
 'Common Chickweed',
 'Fat Hen',
 'Fat Hen',
 'Fat Hen',
 'Fat Hen',
 'Fat Hen',
 'Charlock',
 'Charlock',
 'Fat Hen',
 'Black-grass',
 'Fat Hen',
 'Fat Hen',
 'Fat Hen',
 'Fat Hen',
 'Common wheat',
 'Common wheat',
 'Fat Hen',
 'Black-grass',
 'Scentless Mayweed',
 'Fat Hen',
 'Charlock',
 'Shepherds Purse',
 'Fat Hen',
 'Fat Hen',
 'Fat Hen',
 'Cleavers',
 'Scentless Mayweed',
 'Fat Hen',
 'Fat Hen',
 'Fat Hen',
 'Fat Hen',
 'Fat Hen',
 'Charlock',
 'Fat Hen',
 'Common wheat',
 'Shepherds Purse',
 'Loose Silky-bent',
 'Fat Hen',
 'Small-flowered Cranesbill',
 'Shepherds Purse',
 'Fat Hen',
 'Common wheat',
 'Scentless Mayweed',
 '

In [63]:
sample = pd.read_csv("sample_submission.csv")


In [65]:
# p = model.predict_generator(sample,len(sample))
# p